In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/home/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [2]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    df,
                    ACCESS_TOKEN,
                    accountid,
                    df_use_col_name,
                    fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [3]:
import glob
folder_client_segs="/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/"
list_files_to_upload_PYD=glob.glob(folder_client_segs+"*.csv")
# list_files_to_upload_PYD=[x for x in list_files_to_upload_PYD if "_C.csv" not in x]
list_files_to_upload_PYD.sort()
list_files_to_upload_PYD

['/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audi

In [4]:
list_files_to_upload_PYD

['/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audi

In [ ]:
for file in list_files_to_upload_PYD:
    for fb_account_id in list_account_id:
        audience_name = os.path.basename(file).split(".")[0].replace("removed_Q3_QuarterlyControl_09142020_","2020Spet_")
        description=os.path.basename(file).split(".")[0]+"_"+str(datetime.datetime.now().date())
        df = pd.read_csv(file,dtype=str)
        creatfbaudience(AudienceName="aud_"+audience_name,
                        AudienceDescription=description+", file received from client on 20200915, 107 agile",
                        df=df,
                        df_use_col_name='customersummary_emailhash',
                        ACCESS_TOKEN=jian_token,
                        accountid=fb_account_id)

Audience ID: 23845409902160265
2020-09-16 23:09:19.387586 aud_removed_Q3_QuarterlyControl_09142020_PYD_Active_A uploaded 6473944
Audience ID: 23845409983440265
2020-09-16 23:22:12.683409 aud_removed_Q3_QuarterlyControl_09142020_PYD_Active_B uploaded 6474745
Audience ID: 23845410052540265
2020-09-16 23:34:16.302555 aud_removed_Q3_QuarterlyControl_09142020_PYD_Active_C uploaded 6474337
Audience ID: 23845410148900265
2020-09-16 23:36:47.913082 aud_removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_A uploaded 1174859
Audience ID: 23845410159460265
2020-09-16 23:38:52.645994 aud_removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_B uploaded 1174618
Audience ID: 23845410187740265
2020-09-16 23:40:49.100831 aud_removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_C uploaded 1174524
Audience ID: 23845410223460265
2020-09-16 23:42:39.242091 aud_removed_Q3_QuarterlyControl_09142020_PYD_Signup_NoPurchase_A uploaded 971806
Audience ID: 23845410230810265
2020-09-16 23:44:29.663919 aud_removed_Q3_QuarterlyContr